In [3]:
get_ipython().run_line_magic('matplotlib', 'inline')
# Importing standard Qiskit libraries and configuring account
from qiskit import *
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import numpy as np

In [4]:
from qiskit.providers.aer import noise, QasmSimulator, StatevectorSimulator, UnitarySimulator
from qiskit.providers.aer.noise import NoiseModel


# Loading your IBM Q account(s)
TOKEN ='32df0ea3fc138c8c6122a5e8def5d1a77b954f7d8d463265a1c788100803c5be93d42bfca4b7ce270497c7df1b6004e46493e16e0cf916563f72d06671cc837b'
IBMQ.save_account(TOKEN)
IBMQ.load_account() # Load account from disk
providers = IBMQ.providers()
provider = IBMQ.get_provider(hub ='ibm-q')
print(provider)
print(provider.backends())
backend = provider.get_backend('ibmq_london')



Credentials already present. Set overwrite=True to overwrite.


RequestsApiError: "HTTPSConnectionPool(host='auth.quantum-computing.ibm.com', port=443): Max retries exceeded with url: /api/version (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 405 Method Not Allowed')))"

In [9]:
def state_creation_circuit(index,qubits):
    if index == 1:
        state_create=QuantumCircuit(qubits,qubits)
    elif index ==2:
        state_create=QuantumCircuit(qubits,qubits)
        state_create.x(0)
    elif index == 3:
        state_create=QuantumCircuit(qubits,qubits)
        state_create.x(qubits-1)
        state_create.x(0)
    elif index == 4:
        state_create=QuantumCircuit(qubits,qubits)
        for i in range(qubits):
            state_create.x(i)
    return state_create

countdown={}
noisycounts={}

In [10]:
#This function creates a noise model to apply to a circuit 
# Returns- the amount of counts for each state with noise ML: How does this work??? 
def applyNoise(qc):
    gate_lengths = [
    ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
    ('cx', [1, 0], 678), ('cx', [1, 2], 547), ('cx', [2, 3], 721),
    ('cx', [4, 3], 733), ('cx', [4, 10], 721), ('cx', [5, 4], 800),
    ('cx', [5, 6], 800), ('cx', [5, 9], 895), ('cx', [6, 8], 895),
    ('cx', [7, 8], 640), ('cx', [9, 8], 895), ('cx', [9, 10], 800),
    ('cx', [11, 10], 721), ('cx', [11, 3], 634), ('cx', [12, 2], 773),
    ('cx', [13, 1], 2286), ('cx', [13, 12], 1504), ('cx', [], 800)
   ]#what are the gate lengths needed for? In units of ns
    noise_model = NoiseModel.from_backend(properties, gate_lengths=gate_lengths)
    #noise_model = noise.device.basic_device_noise_model(properties, gate_lengths=gate_lengths)
    #print(noise_model)
    basis_gates = noise_model.basis_gates
    #simulator = Aer.get_backend('qasm_simulator') #what does THIS do? this is already defined like 5 times
    result_noise = execute(qc, backend_sim, shots = 10000,
                          noise_model = noise_model,
                          coupling_map = coupling_map,
                          basis_gates = basis_gates).result()
    counts_noise = result_noise.get_counts(qc)
    return counts_noise

In [11]:
def fidelity(total):
    temp=countdown["V1"]
    value1=(np.absolute(temp["0000"]-.5*total)+np.absolute(temp["0101"]-.5*total))/total
    value1=1-value1
    temp=countdown["V2"]
    value2=(np.absolute(temp["0001"]-.5*total)+np.absolute(temp["0100"]-.5*total))/total
    value2=1-value2
    temp=countdown["V3"]
    value3=(np.absolute(temp["0001"]-.5*total)+np.absolute(temp["0100"]-.5*total))/total
    value3=1-value3
    temp=countdown["V4"]
    value4=(np.absolute(temp["0000"]-.5*total)+np.absolute(temp["0101"]-.5*total))/total
    value4=1-value4
    avg=(value1+value2+value3+value4)/4
    print(avg)

In [ ]:
fidelity(10000)

In [12]:
##Backends options

#takes as argument a circuit, a choice of backend (numbered), and a device name (str)
#if choice ==1, device name is irrelevant
def backend_choice(qc,choice,device_name):
    if choice == 1:
        ##simulator
        backend = Aer.get_backend('qasm_simulator')
        result = execute(qc,backend,shots=10000).result()
        counts = result.get_counts()
        noise = 0
    if choice == 2:
        ##Using real machine as a backend
        provider = IBMQ.get_provider(hub='ibm-q')
        backend_sim = provider.get_backend(device_name)
        properties = device.properties() #will be used to help generate a noise model to run on the simulator
        coupling_map = device.configuration().coupling_map
        result = execute(qc,backend,shots=10000).result()
        counts = result.get_counts()
        noise=applyNoise(qc)#returns noisy state counts data
    if choice == 3:
        ##Using real machines as noise model
        provider = IBMQ.get_provider(hub = 'ibm-q') 
        device = provider.get_backend(device_name) 
        result = execute(qc, backend).result()
        counts=result.get_counts()
        noise=0
    return counts,noise
 


In [13]:
##Get repeater circuit noise and fidelity

for i in range(1,5):
    text="V"+str(i)
    repeater=0
    repeater=state_creation_circuit(i,4)
    repeater.h(0)
    repeater.h(2)
    repeater.cx(0,1)
    repeater.cx(2,3)
    repeater.h(0)
    repeater.h(1)
    repeater.h(2)
    repeater.h(3)
    repeater.cx(1,2)
    repeater.h(3)
    repeater.h(1)
    repeater.h(2)
    repeater.cx(1,2)
    repeater.h(1)
    repeater.h(2)
    repeater.cx(0,1)
    repeater.h(0)
    repeater.h(1)
    repeater.cx(0,1)
    repeater.h(0)
    repeater.h(1)
    repeater.cx(0,1)
    repeater.h(1)
    repeater.cx(1,3)
    repeater.h(1)
    repeater.h(3)
    repeater.cx(0,1)
    repeater.h(0)
    repeater.h(1)
    repeater.cx(0,1)
    repeater.h(0)
    repeater.h(1)
    repeater.cx(0,1)
    repeater.measure(0,0)
    repeater.measure(2,2)
    (counts,noise)=backend_choice(repeater,1,'ibmq_london')
    #result = execute(repeater,backend,shots=10000).result()
    #counts = result.get_counts()
    countdown[text]=counts
    noisycounts[text]=noise

print("fidelity:")
fidelity(10000)

0.98645
None
